In [1]:
from bs4 import BeautifulSoup
import urllib as urllib
from profanity import profanity
from random import shuffle
import json

In [2]:
def make_url(search_string):
    search = search_string.replace(" ", "%20")
    full = "https://www.poetryfoundation.org/search?query=" + search + "&refinement=poems&school-period=1951-present"
    return full

In [3]:
def get_links(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    links = soup.find_all("h2", class_="c-hdgSans c-hdgSans_2")
    for i in range(len(links)):
        links[i] = links[i].a["href"]
    return(links)

In [4]:
def get_poem(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    lines = []
    poem = soup.find("div", class_="o-poem")
    if poem:
        lines = poem.find_all("div")
    out = []
    if len(lines) > 300:
        lines = lines[:201]
    for line in lines:
        poem_line = line.get_text().replace(u'\r ', '').replace(u'\r', '')
        if 'thou ' in poem_line or ' thy ' in poem_line or 'o\'er' in poem_line or ' dost ' in poem_line:
            return []
        if all(ord(char) < 128 for char in poem_line) and not profanity.contains_profanity(poem_line):
            out.append(poem_line)
    return out

In [5]:
def get_poems_from_page(url):
    links = get_links(url)
    poems = []
    for link in links:
        poems.append(get_poem(link))
    return poems

In [6]:
def combine_poems(list1, list2):
    combined = list1 + list2
    shuffle(combined)
    combined = ["\n".join(x) for x in combined]
    combined = "\n".join(combined)
    return combined

In [7]:
def get_all_poems(search_string, num_pages):
    url = make_url(search_string)
    poems = []
    for i in range(1, num_pages + 1):
        page_poems = get_poems_from_page(url + '&page=' + str(i))
        poems += page_poems
    return poems

In [8]:
pain_poems = get_all_poems('pain', 60)

In [9]:
pain_string = ["\n".join(x) for x in pain_poems]
pain_string = "\n".join(pain_string)

In [10]:
with open("angst-data/input3.txt", "w") as text_file:
    text_file.write(pain_string)

In [30]:
def get_angst_poems(num_pages):
    url = 'https://www.poetryfoundation.org/ajax/poems?sort_by=recently_added&topics=6&school-period=1951-present'
    poems = []
    for i in [10]:
        page_link = url + '&page=' + str(i)
        r = urllib.urlopen(page_link).read()
        soup = BeautifulSoup(r, "lxml")
        full_json = soup.find("p")
        print(full_json)
        '''if full_json:
            full_json = full_json.get_text()
            full_json = json.loads(full_json)
            for j in range(0, len(full_json['Entries'])):
                link = full_json['Entries'][j]['link']
                poem = get_poem(link)
                poems.append(poem)'''
    return full_json

In [31]:
angst_poems = get_angst_poems(60)

<p>{"TotalResults":357,"Entries":[{"id":51732,"title":"Cage","author":"By Josephine Miles","snippet":"Through the branches of the Japanese cherry","link":"https:\/\/www.poetryfoundation.org\/poems\/51732\/cage-56d22fab19143","tags":[{"title":"Learning Resource","link":"https:\/\/www.poetryfoundation.org\/poems\/browse#filter_poetry_learning=1&amp;page=1&amp;sort_by=recently_added","iconSlug":"learningResources"}],"categories":[{"id":2,"title":"Living","link":"https:\/\/www.poetryfoundation.org\/poems\/browse#topics=2&amp;page=1&amp;sort_by=recently_added"},{"id":6,"title":"Disappointment &amp; Failure","link":"https:\/\/www.poetryfoundation.org\/poems\/browse#topics=6&amp;page=1&amp;sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\/\/www.poetryfoundation.org\/poems\/browse#topics=38&amp;page=1&amp;sort_by=recently_added"},{"id":43,"title":"Men &amp; Women","link":"https:\/\/www.poetryfoundation.org\/poems\/browse#topics=43&amp;page=1&amp;sort_by=recently_added"}

In [32]:
angst_poems.get_text()

'{"TotalResults":357,"Entries":[{"id":51732,"title":"Cage","author":"By Josephine Miles","snippet":"Through the branches of the Japanese cherry","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51732\\/cage-56d22fab19143","tags":[{"title":"Learning Resource","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_learning=1&page=1&sort_by=recently_added","iconSlug":"learningResources"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"},{"id":45,"title":"Nature

In [ ]:
angst_string = ["\n".join(x) for x in angst_poems]
angst_string = "\n".join(angst_string)

In [ ]:
len(angst_string)

In [ ]:
with open("angst-data/input2.txt", "w") as text_file:
    text_file.write(angst_string)

u'https://www.poetryfoundation.org/poetrymagazine/poems/144809/the-orchestral-audition'

In [277]:
range(1, 10) + [11, 12, 13, 16, 17, 18, 

20

In [35]:
a = r'{"TotalResults":357,"Entries":[{"id":51732,"title":"Cage","author":"By Josephine Miles","snippet":"Through the branches of the Japanese cherry","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51732\\/cage-56d22fab19143","tags":[{"title":"Learning Resource","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_learning=1&page=1&sort_by=recently_added","iconSlug":"learningResources"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":53,"title":"Trees & Flowers","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=53&page=1&sort_by=recently_added"},{"id":278,"title":"Metaphor","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=278&page=1&sort_by=recently_added"},{"id":279,"title":"Imagery","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=279&page=1&sort_by=recently_added"}]},{"id":51632,"title":"Elliptical","author":"By Harryette Mullen","snippet":"They just can’t seem to . . . They should try harder to . . . They ought to be more . . . We all wish they weren’t so . . . They never . . . They always . . . Sometimes they . . . Once in a while they . . . However it is obvious that they . . . Their overall tendency has been . . . The consequences of which have been . . . They don’t appear to understand that . . . If only they would make an effort to . . . But we know how difficult it is for them to . . . Many of them remain unaware of . . . Some who should know better simply refuse to . . . Of course, their perspective has been limited by . . . On the other hand, they obviously feel entitled to . . . Certainly we can’t forget that they . . . Nor can it be denied that they . . . We know that this has had an enormous impact on their . . . Nevertheless their behavior strikes us as . . . Our interactions unfortunately have been . . .","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51632\\/elliptical","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":40,"title":"Friends & Enemies","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=40&page=1&sort_by=recently_added"},{"id":65,"title":"Arts & Sciences","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=65&page=1&sort_by=recently_added"},{"id":68,"title":"Language & Linguistics","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=68&page=1&sort_by=recently_added"},{"id":281,"title":"Prose Poem","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=281&page=1&sort_by=recently_added"}]},{"id":51872,"title":"Human Cylinders","author":"By Mina Loy","snippet":"The human cylinders","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51872\\/human-cylinders","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"}]},{"id":51636,"title":"Suite for Emily","author":"By Lynda Hull","snippet":"1. The Letter","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51636\\/suite-for-emily","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":4,"title":"Coming of Age","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=4&page=1&sort_by=recently_added"},{"id":5,"title":"Death","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=5&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":7,"title":"Growing Old","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=7&page=1&sort_by=recently_added"},{"id":8,"title":"Health & Illness","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=8&page=1&sort_by=recently_added"},{"id":10,"title":"Life Choices","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=10&page=1&sort_by=recently_added"},{"id":15,"title":"Sorrow & Grieving","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=15&page=1&sort_by=recently_added"},{"id":16,"title":"The Body","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=16&page=1&sort_by=recently_added"},{"id":17,"title":"The Mind","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=17&page=1&sort_by=recently_added"},{"id":18,"title":"Time & Brevity","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=18&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":40,"title":"Friends & Enemies","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=40&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":48,"title":"Landscapes & Pastorals","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=48&page=1&sort_by=recently_added"},{"id":55,"title":"Winter","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=55&page=1&sort_by=recently_added"},{"id":56,"title":"Religion","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=56&page=1&sort_by=recently_added"},{"id":59,"title":"Faith & Doubt","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=59&page=1&sort_by=recently_added"},{"id":60,"title":"God & the Divine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=60&page=1&sort_by=recently_added"},{"id":64,"title":"The Spiritual","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=64&page=1&sort_by=recently_added"},{"id":77,"title":"Social Commentaries","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=77&page=1&sort_by=recently_added"},{"id":78,"title":"Cities & Urban Life","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=78&page=1&sort_by=recently_added"},{"id":80,"title":"Crime & Punishment","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=80&page=1&sort_by=recently_added"},{"id":81,"title":"Gender & Sexuality","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=81&page=1&sort_by=recently_added"},{"id":82,"title":"History & Politics","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=82&page=1&sort_by=recently_added"},{"id":88,"title":"Mythology & Folklore","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=88&page=1&sort_by=recently_added"},{"id":90,"title":"Ghosts & the Supernatural","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=90&page=1&sort_by=recently_added"}]},{"id":52177,"title":"[What horror to awake at night]","author":"By Lorine Niedecker","snippet":"What horror to awake at night","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/52177\\/what-horror-to-awake-at-night","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":18,"title":"Time & Brevity","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=18&page=1&sort_by=recently_added"},{"id":77,"title":"Social Commentaries","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=77&page=1&sort_by=recently_added"}]},{"id":51966,"title":"Applied Geometry","author":"By Russell Libby","snippet":"Applied geometry, ","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51966\\/applied-geometry","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":53,"title":"Trees & Flowers","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=53&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":51853,"title":"Cutting Hair","author":"By Minnie Bruce Pratt","snippet":"She pays attention to the hair, not her fingers, and cuts herself ","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/51853\\/cutting-hair","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":8,"title":"Health & Illness","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=8&page=1&sort_by=recently_added"},{"id":16,"title":"The Body","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=16&page=1&sort_by=recently_added"},{"id":30,"title":"Activities","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=30&page=1&sort_by=recently_added"},{"id":34,"title":"Jobs & Working","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=34&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":49497,"title":"Drought","author":"By Felecia Caton Garcia","snippet":"Try to remember: things go wrong in spite of it all. ","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/49497\\/drought-56d22ba447ed1","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":13,"title":"Parenthood","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=13&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":39,"title":"Family & Ancestors","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=39&page=1&sort_by=recently_added"}]},{"id":49278,"title":"How Are You Doing?","author":"By Rick Snyder","snippet":"As much as you deserve it,","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/49278\\/how-are-you-doing","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":30,"title":"Activities","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=30&page=1&sort_by=recently_added"},{"id":34,"title":"Jobs & Working","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=34&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"},{"id":280,"title":"Refrain","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=280&page=1&sort_by=recently_added"}]},{"id":50878,"title":"Today’s News","author":"By David Tucker","snippet":"A slow news day, but I did like the obit about the butcher\\u00a0\\u00a0\\u00a0","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/50878\\/todays-news","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":18,"title":"Time & Brevity","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=18&page=1&sort_by=recently_added"},{"id":30,"title":"Activities","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=30&page=1&sort_by=recently_added"},{"id":34,"title":"Jobs & Working","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=34&page=1&sort_by=recently_added"},{"id":77,"title":"Social Commentaries","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=77&page=1&sort_by=recently_added"},{"id":83,"title":"Money & Economics","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=83&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":52261,"title":"A Gift","author":"By Kathryn Starbuck","snippet":"Who is that creature","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/52261\\/a-gift-56d23095c2901","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":15,"title":"Sorrow & Grieving","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=15&page=1&sort_by=recently_added"},{"id":20,"title":"Love","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=20&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":43,"title":"Men & Women","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=43&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":51996,"title":"Feather in Bas-Relief","author":"By Allen Edwin Butt","snippet":"Words without much use ","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/51996\\/feather-in-bas-relief","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"},{"title":"Related Audio","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_audio=1&page=1&sort_by=recently_added","iconSlug":"tagAudio"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":46,"title":"Animals","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=46&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":51986,"title":"I Hate","author":"By C. K. Williams","snippet":"I hate how this unsummoned sigh-sound, sob-sound, ","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/51986\\/i-hate","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":15,"title":"Sorrow & Grieving","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=15&page=1&sort_by=recently_added"},{"id":18,"title":"Time & Brevity","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=18&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":49591,"title":""Love of My Flesh, Living Death"","author":"By Lorna Dee Cervantes","snippet":"Once I wasn’t always so plain.","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/49591\\/love-of-my-flesh-living-death","tags":[],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":20,"title":"Love","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=20&page=1&sort_by=recently_added"},{"id":27,"title":"Realistic & Complicated","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=27&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":46,"title":"Animals","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=46&page=1&sort_by=recently_added"},{"id":48,"title":"Landscapes & Pastorals","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=48&page=1&sort_by=recently_added"},{"id":263,"title":"Sonnet","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=263&page=1&sort_by=recently_added"},{"id":276,"title":"Allusion","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=276&page=1&sort_by=recently_added"}]},{"id":40463,"title":"“Find Work”","author":"By Rhina P. Espaillat","snippet":"My mother’s mother, widowed very young","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/40463\\/find-work","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"},{"title":"Suitable for Children","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_children=1&page=1&sort_by=recently_added","iconSlug":"children"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":7,"title":"Growing Old","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=7&page=1&sort_by=recently_added"},{"id":12,"title":"Midlife","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=12&page=1&sort_by=recently_added"},{"id":13,"title":"Parenthood","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=13&page=1&sort_by=recently_added"},{"id":15,"title":"Sorrow & Grieving","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=15&page=1&sort_by=recently_added"},{"id":38,"title":"Relationships","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=38&page=1&sort_by=recently_added"},{"id":39,"title":"Family & Ancestors","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=39&page=1&sort_by=recently_added"},{"id":263,"title":"Sonnet","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=263&page=1&sort_by=recently_added"}]},{"id":40361,"title":"kommentar\\/comment","author":"By Ernst Jandl","snippet":"that never ","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/40361\\/kommentar-comment","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":65,"title":"Arts & Sciences","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=65&page=1&sort_by=recently_added"},{"id":73,"title":"Poetry & Poets","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=73&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]},{"id":40284,"title":"Madmen","author":"By Billy Collins","snippet":"They say you can jinx a poem","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/40284\\/madmen","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":65,"title":"Arts & Sciences","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=65&page=1&sort_by=recently_added"},{"id":67,"title":"Humor & Satire","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=67&page=1&sort_by=recently_added"},{"id":70,"title":"Painting & Sculpture","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=70&page=1&sort_by=recently_added"},{"id":73,"title":"Poetry & Poets","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=73&page=1&sort_by=recently_added"},{"id":254,"title":"Elegy","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=254&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"},{"id":278,"title":"Metaphor","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=278&page=1&sort_by=recently_added"}]},{"id":40760,"title":"The Men","author":"By Pablo Neruda","snippet":"I\'m Ramón González Barbagelata from anywhere,","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/40760\\/the-men","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":30,"title":"Activities","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=30&page=1&sort_by=recently_added"},{"id":37,"title":"Travels & Journeys","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=37&page=1&sort_by=recently_added"},{"id":77,"title":"Social Commentaries","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=77&page=1&sort_by=recently_added"},{"id":79,"title":"Class","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=79&page=1&sort_by=recently_added"},{"id":83,"title":"Money & Economics","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=83&page=1&sort_by=recently_added"},{"id":251,"title":"Pastoral","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=251&page=1&sort_by=recently_added"},{"id":253,"title":"Dramatic Monologue","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=253&page=1&sort_by=recently_added"},{"id":279,"title":"Imagery","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=279&page=1&sort_by=recently_added"}]},{"id":41445,"title":"Minor Poet","author":"By Bill Sweeney","snippet":"His last composed poem, \\"Over My Head,\\" \\r","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/41445\\/minor-poet","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":49,"title":"Seas, Rivers, & Streams","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=49&page=1&sort_by=recently_added"},{"id":65,"title":"Arts & Sciences","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=65&page=1&sort_by=recently_added"},{"id":73,"title":"Poetry & Poets","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=73&page=1&sort_by=recently_added"},{"id":77,"title":"Social Commentaries","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=77&page=1&sort_by=recently_added"},{"id":78,"title":"Cities & Urban Life","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=78&page=1&sort_by=recently_added"},{"id":277,"title":"Simile","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=277&page=1&sort_by=recently_added"},{"id":279,"title":"Imagery","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=279&page=1&sort_by=recently_added"}]},{"id":41063,"title":"Postpartum Blues","author":"By Elton Glaser","snippet":"Soon the glass angel must be \\r","link":"https:\\/\\/www.poetryfoundation.org\\/poetrymagazine\\/poems\\/41063\\/postpartum-blues","tags":[{"title":"Appeared in Poetry Magazine","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#filter_poetry_first=1&page=1&sort_by=recently_added","iconSlug":"poetryMagazine"}],"categories":[{"id":2,"title":"Living","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=2&page=1&sort_by=recently_added"},{"id":5,"title":"Death","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=5&page=1&sort_by=recently_added"},{"id":6,"title":"Disappointment & Failure","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=6&page=1&sort_by=recently_added"},{"id":13,"title":"Parenthood","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=13&page=1&sort_by=recently_added"},{"id":15,"title":"Sorrow & Grieving","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=15&page=1&sort_by=recently_added"},{"id":45,"title":"Nature","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=45&page=1&sort_by=recently_added"},{"id":55,"title":"Winter","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=55&page=1&sort_by=recently_added"},{"id":259,"title":"Free Verse","link":"https:\\/\\/www.poetryfoundation.org\\/poems\\/browse#topics=259&page=1&sort_by=recently_added"}]}],"Filters":[{"id":null,"name":"Topics","slug":"topics","isTrigger":false,"categories":[{"id":1,"name":"Subjects","slug":"subjects","isTrigger":false,"key":"topics","children":[{"id":20,"name":"Love","slug":"love","isTrigger":true,"children":[{"id":23,"name":"Desire","slug":"desire","isTrigger":true},{"id":25,"name":"Heartache & Loss","slug":"heartache-loss","isTrigger":true},{"id":27,"name":"Realistic & Complicated","slug":"realistic-complicated","isTrigger":true},{"id":28,"name":"Romantic Love","slug":"romantic-love","isTrigger":true},{"id":22,"name":"Classic Love","slug":"classic-love","isTrigger":true},{"id":26,"name":"Infatuation & Crushes","slug":"infatuation-crushes","isTrigger":true},{"id":29,"name":"Unrequited Love","slug":"unrequited-love","isTrigger":true},{"id":21,"name":"Break-ups & Vexed Love","slug":"break-ups-vexed-love","isTrigger":true},{"id":24,"name":"First Love","slug":"first-love","isTrigger":true}]},{"id":45,"name":"Nature","slug":"nature","isTrigger":true,"children":[{"id":55,"name":"Winter","slug":"winter","isTrigger":true},{"id":50,"name":"Spring","slug":"spring","isTrigger":true},{"id":52,"name":"Summer","slug":"summer","isTrigger":true},{"id":47,"name":"Fall","slug":"fall","isTrigger":true},{"id":48,"name":"Landscapes & Pastorals","slug":"landscapes-pastorals","isTrigger":true},{"id":54,"name":"Weather","slug":"weather","isTrigger":true},{"id":53,"name":"Trees & Flowers","slug":"trees-flowers","isTrigger":true},{"id":51,"name":"Stars, Planets, Heavens","slug":"stars-planets-heavens","isTrigger":true},{"id":49,"name":"Seas, Rivers, & Streams","slug":"seas-rivers-streams","isTrigger":true},{"id":46,"name":"Animals","slug":"animals","isTrigger":true}]},{"id":77,"name":"Social Commentaries","slug":"social-commentaries","isTrigger":true,"children":[{"id":82,"name":"History & Politics","slug":"history-politics","isTrigger":true},{"id":87,"name":"War & Conflict","slug":"war-conflict","isTrigger":true},{"id":80,"name":"Crime & Punishment","slug":"crime-punishment","isTrigger":true},{"id":83,"name":"Money & Economics","slug":"money-economics","isTrigger":true},{"id":85,"name":"Race & Ethnicity","slug":"race-ethnicity","isTrigger":true},{"id":81,"name":"Gender & Sexuality","slug":"gender-sexuality","isTrigger":true},{"id":79,"name":"Class","slug":"class","isTrigger":true},{"id":84,"name":"Popular Culture","slug":"popular-culture","isTrigger":true},{"id":78,"name":"Cities & Urban Life","slug":"cities-urban-life","isTrigger":true},{"id":86,"name":"Town & Country Life","slug":"town-country-life","isTrigger":true}]},{"id":56,"name":"Religion","slug":"religion","isTrigger":true,"children":[{"id":59,"name":"Faith & Doubt","slug":"faith-doubt","isTrigger":true},{"id":60,"name":"God & the Divine","slug":"god-the-divine","isTrigger":true},{"id":62,"name":"Judaism","slug":"judaism","isTrigger":true},{"id":57,"name":"Buddhism","slug":"buddhism","isTrigger":true},{"id":61,"name":"Islam","slug":"islam","isTrigger":true},{"id":58,"name":"Christianity","slug":"christianity","isTrigger":true},{"id":63,"name":"Other Religions","slug":"other-religions","isTrigger":true},{"id":64,"name":"The Spiritual","slug":"the-spiritual","isTrigger":true}]},{"id":2,"name":"Living","slug":"living","isTrigger":true,"children":[{"id":3,"name":"Birth & Birthdays","slug":"birth-birthdays","isTrigger":true},{"id":9,"name":"Infancy","slug":"infancy","isTrigger":true},{"id":19,"name":"Youth","slug":"youth","isTrigger":true},{"id":4,"name":"Coming of Age","slug":"coming-of-age","isTrigger":true},{"id":11,"name":"Marriage & Companionship","slug":"marriage-companionship","isTrigger":true},{"id":13,"name":"Parenthood","slug":"parenthood","isTrigger":true},{"id":14,"name":"Separation & Divorce","slug":"separation-divorce","isTrigger":true},{"id":12,"name":"Midlife","slug":"midlife","isTrigger":true},{"id":7,"name":"Growing Old","slug":"growing-old","isTrigger":true},{"id":8,"name":"Health & Illness","slug":"health-illness","isTrigger":true},{"id":6,"name":"Disappointment & Failure","slug":"disappointment-failure","isTrigger":true},{"id":5,"name":"Death","slug":"death","isTrigger":true},{"id":15,"name":"Sorrow & Grieving","slug":"sorrow-grieving","isTrigger":true},{"id":10,"name":"Life Choices","slug":"life-choices","isTrigger":true},{"id":16,"name":"The Body","slug":"the-body","isTrigger":true},{"id":17,"name":"The Mind","slug":"the-mind","isTrigger":true},{"id":18,"name":"Time & Brevity","slug":"time-brevity","isTrigger":true}]},{"id":38,"name":"Relationships","slug":"relationships","isTrigger":true,"children":[{"id":43,"name":"Men & Women","slug":"men-women","isTrigger":true},{"id":40,"name":"Friends & Enemies","slug":"friends-enemies","isTrigger":true},{"id":41,"name":"Gay, Lesbian, Queer","slug":"gay-lesbian-queer","isTrigger":true},{"id":42,"name":"Home Life","slug":"home-life","isTrigger":true},{"id":39,"name":"Family & Ancestors","slug":"family-ancestors","isTrigger":true},{"id":44,"name":"Pets","slug":"pets","isTrigger":true}]},{"id":30,"name":"Activities","slug":"activities","isTrigger":true,"children":[{"id":35,"name":"School & Learning","slug":"school-learning","isTrigger":true},{"id":34,"name":"Jobs & Working","slug":"jobs-working","isTrigger":true},{"id":37,"name":"Travels & Journeys","slug":"travels-journeys","isTrigger":true},{"id":36,"name":"Sports & Outdoor Activities","slug":"sports-outdoor-activities","isTrigger":true},{"id":32,"name":"Gardening","slug":"gardening","isTrigger":true},{"id":31,"name":"Eating & Drinking","slug":"eating-drinking","isTrigger":true},{"id":33,"name":"Indoor Activities","slug":"indoor-activities","isTrigger":true}]},{"id":65,"name":"Arts & Sciences","slug":"arts-sciences","isTrigger":true,"children":[{"id":71,"name":"Philosophy","slug":"philosophy","isTrigger":true},{"id":75,"name":"Sciences","slug":"sciences","isTrigger":true},{"id":74,"name":"Reading & Books","slug":"reading-books","isTrigger":true},{"id":73,"name":"Poetry & Poets","slug":"poetry-poets","isTrigger":true},{"id":67,"name":"Humor & Satire","slug":"humor-satire","isTrigger":true},{"id":69,"name":"Music","slug":"music","isTrigger":true},{"id":76,"name":"Theater & Dance","slug":"theater-dance","isTrigger":true},{"id":70,"name":"Painting & Sculpture","slug":"painting-sculpture","isTrigger":true},{"id":66,"name":"Architecture & Design","slug":"architecture-design","isTrigger":true},{"id":72,"name":"Photography & Film","slug":"photography-film","isTrigger":true},{"id":68,"name":"Language & Linguistics","slug":"language-linguistics","isTrigger":true}]},{"id":88,"name":"Mythology & Folklore","slug":"mythology-folklore","isTrigger":true,"children":[{"id":90,"name":"Ghosts & the Supernatural","slug":"ghosts-the-supernatural","isTrigger":true},{"id":93,"name":"Horror","slug":"horror","isTrigger":true},{"id":92,"name":"Heroes & Patriotism","slug":"heroes-patriotism","isTrigger":true},{"id":91,"name":"Greek & Roman Mythology","slug":"greek-roman-mythology","isTrigger":true},{"id":89,"name":"Fairy-tales & Legends","slug":"fairy-tales-legends","isTrigger":true}]}]},{"id":94,"name":"Occasions","slug":"occasions","isTrigger":false,"key":"topics","children":[{"id":95,"name":"Anniversary","slug":"anniversary","isTrigger":true},{"id":96,"name":"Birth","slug":"birth","isTrigger":true},{"id":97,"name":"Birthdays","slug":"birthdays","isTrigger":true},{"id":98,"name":"Engagement","slug":"engagement","isTrigger":true},{"id":99,"name":"Farewells & Good Luck","slug":"farewells-good-luck","isTrigger":true},{"id":100,"name":"Funerals","slug":"funerals","isTrigger":true},{"id":101,"name":"Get Well & Recovery","slug":"get-well-recovery","isTrigger":true},{"id":102,"name":"Graduation","slug":"graduation","isTrigger":true},{"id":103,"name":"Gratitude & Apologies","slug":"gratitude-apologies","isTrigger":true},{"id":104,"name":"Toasts & Celebrations","slug":"toasts-celebrations","isTrigger":true},{"id":105,"name":"Weddings","slug":"weddings","isTrigger":true}]},{"id":106,"name":"Holidays","slug":"holidays","isTrigger":false,"key":"topics","children":[{"id":107,"name":"Christmas","slug":"christmas","isTrigger":true},{"id":108,"name":"Cinco de Mayo","slug":"cinco-de-mayo","isTrigger":true},{"id":109,"name":"Easter","slug":"easter","isTrigger":true},{"id":110,"name":"Father\'s Day","slug":"father-s-day","isTrigger":true},{"id":111,"name":"Halloween","slug":"halloween","isTrigger":true},{"id":112,"name":"Hanukkah","slug":"hanukah","isTrigger":true},{"id":113,"name":"Independence Day","slug":"independence-day","isTrigger":true},{"id":114,"name":"Kwanzaa","slug":"kwanzaa","isTrigger":true},{"id":115,"name":"Labor Day","slug":"labor-day","isTrigger":true},{"id":116,"name":"Memorial Day","slug":"memorial-day","isTrigger":true},{"id":117,"name":"Mother\'s Day","slug":"mother-s-day","isTrigger":true},{"id":118,"name":"New Year","slug":"new-year","isTrigger":true},{"id":119,"name":"Passover","slug":"passover","isTrigger":true},{"id":120,"name":"Ramadan","slug":"ramadan","isTrigger":true},{"id":121,"name":"Rosh Hashanah","slug":"rosh-hashanah","isTrigger":true},{"id":122,"name":"September 11th","slug":"september-11th","isTrigger":true},{"id":123,"name":"St. Patrick\'s Day","slug":"st-patrick-s-day","isTrigger":true},{"id":124,"name":"Thanksgiving","slug":"thanksgiving","isTrigger":true},{"id":125,"name":"Valentine\'s Day","slug":"valentine-s-day","isTrigger":true},{"id":126,"name":"Yom Kippur","slug":"yom-kippur","isTrigger":true}]}]},{"id":null,"name":"Forms","slug":"forms","isTrigger":false,"categories":[{"id":245,"name":"Verse Forms","slug":"verse-forms","isTrigger":false,"key":"forms","children":[{"id":257,"name":"Ballad","slug":"ballad","isTrigger":true},{"id":260,"name":"Haiku","slug":"haiku","isTrigger":true},{"id":261,"name":"Limerick","slug":"limerick","isTrigger":true},{"id":262,"name":"Sestina","slug":"sestina","isTrigger":true},{"id":263,"name":"Sonnet","slug":"sonnet","isTrigger":true},{"id":266,"name":"Villanelle","slug":"villanelle","isTrigger":true},{"id":272,"name":"Pantoum","slug":"pantoum","isTrigger":true},{"id":290,"name":"Ghazal","slug":"ghazal","isTrigger":true}]},{"id":246,"name":"Stanza Forms","slug":"stanza-forms","isTrigger":false,"key":"forms","children":[{"id":258,"name":"Couplet","slug":"couplet","isTrigger":true},{"id":264,"name":"Ottava Rima","slug":"ottava-rima","isTrigger":true},{"id":265,"name":"Terza Rima","slug":"terza-rima","isTrigger":true},{"id":282,"name":"Rhymed Stanza","slug":"rhymed-stanza","isTrigger":true},{"id":284,"name":"Mixed","slug":"mixed","isTrigger":true},{"id":292,"name":"Tercet","slug":"tercet","isTrigger":true},{"id":293,"name":"Quatrain","slug":"quatrain","isTrigger":true}]},{"id":247,"name":"Meters","slug":"meters","isTrigger":false,"key":"forms","children":[{"id":259,"name":"Free Verse","slug":"free-verse","isTrigger":true},{"id":268,"name":"Blank Verse","slug":"blank-verse","isTrigger":true},{"id":271,"name":"Syllabic","slug":"syllabic","isTrigger":true},{"id":283,"name":"Common Measure","slug":"common-measure","isTrigger":true}]},{"id":248,"name":"Techniques","slug":"techniques","isTrigger":false,"key":"forms","children":[{"id":270,"name":"Epigraph","slug":"epigraph","isTrigger":true},{"id":273,"name":"Assonance","slug":"assonance","isTrigger":true},{"id":274,"name":"Consonance","slug":"consonance","isTrigger":true},{"id":275,"name":"Alliteration","slug":"alliteration","isTrigger":true},{"id":276,"name":"Allusion","slug":"allusion","isTrigger":true},{"id":277,"name":"Simile","slug":"simile","isTrigger":true},{"id":278,"name":"Metaphor","slug":"metaphor","isTrigger":true},{"id":279,"name":"Imagery","slug":"imagery","isTrigger":true},{"id":280,"name":"Refrain","slug":"refrain","isTrigger":true},{"id":287,"name":"Aphorism","slug":"aphorism","isTrigger":true},{"id":294,"name":"Persona","slug":"persona","isTrigger":true},{"id":296,"name":"Imagist","slug":"imagist-1","isTrigger":true},{"id":297,"name":"Confessional","slug":"confessional-1","isTrigger":true},{"id":298,"name":"Symbolist","slug":"symbolist","isTrigger":true}]},{"id":249,"name":"Types\\/Modes","slug":"types-modes","isTrigger":false,"key":"forms","children":[{"id":250,"name":"Ode","slug":"ode","isTrigger":true},{"id":251,"name":"Pastoral","slug":"pastoral","isTrigger":true},{"id":252,"name":"Aubade","slug":"aubade","isTrigger":true},{"id":253,"name":"Dramatic Monologue","slug":"dramatic-monologue","isTrigger":true},{"id":254,"name":"Elegy","slug":"elegy","isTrigger":true},{"id":255,"name":"Epistle","slug":"epistle","isTrigger":true},{"id":256,"name":"Epithalamion","slug":"epithalamion","isTrigger":true},{"id":267,"name":"Concrete or Pattern Poetry","slug":"concrete-or-pattern-poetry","isTrigger":true},{"id":269,"name":"Epigram","slug":"epigram","isTrigger":true},{"id":281,"name":"Prose Poem","slug":"prose-poem","isTrigger":true},{"id":285,"name":"Series\\/Sequence","slug":"series-sequence","isTrigger":true},{"id":286,"name":"Visual Poetry","slug":"visual-poetry","isTrigger":true},{"id":288,"name":"Ars Poetica","slug":"ars-poetica","isTrigger":true},{"id":289,"name":"Ekphrasis","slug":"ekphrasis","isTrigger":true},{"id":291,"name":"Epic","slug":"epic","isTrigger":true},{"id":295,"name":"Nursery Rhymes","slug":"nursery-rhymes","isTrigger":true}]}]},{"id":"148","name":"School\\/Period","slug":"school-period","isTrigger":false,"isFlat":true,"key":"school-period","categories":[{"id":"pre-1550","name":"Pre-1550","slug":"pre-1550","isTrigger":false,"key":"school-period","children":[{"id":"pre-1550","name":"Pre-1550","slug":"pre-1550","isTrigger":true},{"id":158,"name":"Middle English","slug":"middle-english","isTrigger":true}]},{"id":"1550-1780","name":"1550-1780","slug":"1550-1780","isTrigger":false,"key":"school-period","children":[{"id":"1550-1780","name":"1550-1780","slug":"1550-1780","isTrigger":true},{"id":149,"name":"Augustan","slug":"augustan","isTrigger":true},{"id":163,"name":"Renaissance","slug":"renaissance","isTrigger":true}]},{"id":"1781-1900","name":"1781-1900","slug":"1781-1900","isTrigger":false,"key":"school-period","children":[{"id":"1781-1900","name":"1781-1900","slug":"1781-1900","isTrigger":true},{"id":164,"name":"Romantic","slug":"romantic","isTrigger":true},{"id":165,"name":"Victorian","slug":"victorian","isTrigger":true}]},{"id":"1901-1950","name":"1901-1950","slug":"1901-1950","isTrigger":false,"key":"school-period","children":[{"id":"1901-1950","name":"1901-1950","slug":"1901-1950","isTrigger":true},{"id":153,"name":"Fugitive","slug":"fugitive","isTrigger":true},{"id":154,"name":"Georgian","slug":"georgian","isTrigger":true},{"id":155,"name":"Harlem Renaissance","slug":"harlem-renaissance","isTrigger":true},{"id":156,"name":"Imagist","slug":"imagist","isTrigger":true},{"id":159,"name":"Modern","slug":"modern","isTrigger":true},{"id":162,"name":"Objectivist","slug":"objectivist","isTrigger":true}]},{"id":"1951-present","name":"1951-Present","slug":"1951-present","isTrigger":false,"key":"school-period","children":[{"id":"1951-present","name":"1951-Present","slug":"1951-present","isTrigger":true},{"id":150,"name":"Beat","slug":"beat","isTrigger":true},{"id":304,"name":"Black Arts Movement","slug":"black-arts-movement","isTrigger":true},{"id":151,"name":"Black Mountain","slug":"black-mountain","isTrigger":true},{"id":152,"name":"Confessional","slug":"confessional","isTrigger":true},{"id":157,"name":"Language Poetry","slug":"language-poetry","isTrigger":true},{"id":160,"name":"New York School","slug":"new-york-school","isTrigger":true},{"id":161,"name":"New York School (2nd Generation)","slug":"new-york-school-2nd-generation\\"","isTrigger":true}]}]},{"id":null,"name":"Poet\'s Region","slug":"region","isTrigger":false,"categories":[{"id":null,"name":"Poet\'s Region","slug":"poets-region","isTrigger":false,"key":"region","children":[{"id":167,"name":"Africa","slug":"africa","isTrigger":true},{"id":168,"name":"Asia, East","slug":"asia-east","isTrigger":true},{"id":169,"name":"Asia, South","slug":"asia-south","isTrigger":true},{"id":303,"name":"Asia, Southeast","slug":"asia-southeast","isTrigger":true},{"id":170,"name":"Australia","slug":"australia","isTrigger":true},{"id":171,"name":"Canada","slug":"canada","isTrigger":true},{"id":172,"name":"Caribbean","slug":"caribbean","isTrigger":true},{"id":174,"name":"Eastern Europe","slug":"eastern-europe","isTrigger":true},{"id":175,"name":"England","slug":"england","isTrigger":true},{"id":176,"name":"France","slug":"france","isTrigger":true},{"id":177,"name":"Germany","slug":"germany","isTrigger":true},{"id":178,"name":"Greece","slug":"greece","isTrigger":true},{"id":179,"name":"Ireland & Northern Ireland","slug":"ireland-northern-ireland","isTrigger":true},{"id":180,"name":"Israel","slug":"israel","isTrigger":true},{"id":181,"name":"Italy","slug":"italy","isTrigger":true},{"id":183,"name":"Latin America","slug":"latin-america","isTrigger":true},{"id":184,"name":"Mexico","slug":"mexico","isTrigger":true},{"id":185,"name":"Middle East","slug":"middle-east","isTrigger":true},{"id":301,"name":"Pacific Islands","slug":"pacific-islands","isTrigger":true},{"id":186,"name":"Poland","slug":"poland","isTrigger":true},{"id":187,"name":"Russia","slug":"russia","isTrigger":true},{"id":188,"name":"Scandinavia","slug":"scandinavia","isTrigger":true},{"id":189,"name":"Scotland","slug":"scotland","isTrigger":true},{"id":191,"name":"Spain","slug":"spain","isTrigger":true},{"id":192,"name":"U.S., Mid-Atlantic","slug":"u-s-mid-atlantic","isTrigger":true},{"id":193,"name":"U.S., Midwestern","slug":"u-s-midwestern","isTrigger":true},{"id":194,"name":"U.S., New England","slug":"u-s-new-england","isTrigger":true},{"id":195,"name":"U.S., Northwestern","slug":"u-s-northwestern","isTrigger":true},{"id":196,"name":"U.S., Southern","slug":"u-s-southern","isTrigger":true},{"id":197,"name":"U.S., Southwestern","slug":"u-s-southwestern","isTrigger":true},{"id":198,"name":"U.S., Western","slug":"u-s-western","isTrigger":true},{"id":199,"name":"Wales","slug":"wales","isTrigger":true}]}]}]}'

In [36]:
a_json = json.loads(a)

JSONDecodeError: Expecting ',' delimiter: line 1 column 18976 (char 18975)